In [6]:
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw, ImageFont


def _get_label_id(label):  
    """  
    将标签映射到ID  
    """  
    label_map = {  
        'clickable': 0,
        'Clickable': 0,
        'level_0_clickable':0,
        'level_1_clickable':0,
        'selectable': 1,
        'Selectable': 1,
        'level_0_selectable':1,
        'level_1_selectable':1,
        'scrollable': 2,
        'Scrollable': 2,
        'level_0_scrollable':2,
        'level_1_scrollable':2,
        'disabled': 3,
        'Disabled': 3,
        'level_0_disabled':3,
        'level_1_disabled':3,
        'level_0':4,
        'level_1':4,
        'level_2':4,
        'root':4,
        'Root':4,
    }  
    return label_map.get(label, -1)  

def visualize_bbox(image_path):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # 解析 XML 文件
    xml_path = image_path.replace('.png', '.xml')
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # 获取 bndbox 和 name
    for obj in root.findall('object'):
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        label = _get_label_id(name)
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        if label == 0:  # clickable
            color = 'red'
            font_size = 20
        elif label == 1:  # selectable
            color = 'green'
            font_size = 20
        elif label == 2:  # scrollable
            color = 'blue'
            font_size = 20
        elif label == 3:  # disabled
            color = 'gray'
            font_size = 20
        elif label == 4:
            color = 'black'
            font_size = 20
        # 绘制边框
        draw.rectangle([xmin, ymin, xmax, ymax], outline=color, width=2)
        # 打印名称
        draw.text((xmin, ymin), name, fill=color, font_size=font_size)

    # 保存或显示结果
    # image.show()  # 显示图片
    save_path = image_path.replace('.png', '_annotated.png')
    image.save(save_path)  # 保存图片

import os
dataset_dir = '/ssd/xiaxinyuan/code/hmwk/CV_Final_Project/v1/coredraw'
for image_path in os.listdir(dataset_dir):
    if image_path.endswith('.png') and 'annotated' not in image_path:
        visualize_bbox(os.path.join(dataset_dir, image_path))


FileNotFoundError: [Errno 2] No such file or directory: '/ssd/xiaxinyuan/code/hmwk/CV_Final_Project/v1/coredraw/frame_1_annotated.xml'

In [8]:
import sys
sys.path.append('/ssd/xiaxinyuan/code/hmwk/CV_Final_Project/thirdparty')
sys.path.append('/ssd/xiaxinyuan/code/hmwk/CV_Final_Project/thirdparty/GroundingDINO')
sys.path.append('/ssd/xiaxinyuan/code/hmwk/CV_Final_Project/thirdparty/GroundingDINO/groundingdino')
print(sys.path)
from thirdparty.GroundingDINO.groundingdino.models import build_model

model = build_model()


['/ssd/xiaxinyuan/code/hmwk/CV_Final_Project', '/home/xiaxinyuan/.conda/envs/modelscope/lib/python37.zip', '/home/xiaxinyuan/.conda/envs/modelscope/lib/python3.7', '/home/xiaxinyuan/.conda/envs/modelscope/lib/python3.7/lib-dynload', '', '/home/xiaxinyuan/.conda/envs/modelscope/lib/python3.7/site-packages', '/home/xiaxinyuan/.conda/envs/modelscope/lib/python3.7/site-packages/IPython/extensions', '/home/xiaxinyuan/.ipython', '/ssd/xiaxinyuan/code/hmwk/CV_Final_Project/thirdparty', '/ssd/xiaxinyuan/code/hmwk/CV_Final_Project/thirdparty/GroundingDINO', '/ssd/xiaxinyuan/code/hmwk/CV_Final_Project/thirdparty/GroundingDINO/groundingdino']


/ssd/xiaxinyuan/code/hmwk/CV_Final_Project/thirdparty/GroundingDINO/groundingdino/models/GroundingDINO/ms_deform_attn.py:33: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn('Failed to load custom C++ ops. Running on CPU mode Only!')


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│   5 print(sys.path)                                                                              │
│   6 from thirdparty.GroundingDINO.groundingdino.models import build_model                        │
│   7                                                                                              │
│ ❱ 8 model = build_model()                                                                        │
│   9                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: build_model() missing 1 required positional argument: 'args'

In [10]:
from transformers import AutoProcessor, GroundingDinoForObjectDetection
from PIL import Image
import requests
import torch
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = "a cat."

processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-tiny")
model = GroundingDinoForObjectDetection.from_pretrained("IDEA-Research/grounding-dino-tiny")

inputs = processor(images=image, text=text, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
target_sizes = torch.tensor([image.size[::-1]])
results = processor.image_processor.post_process_object_detection(
    outputs, threshold=0.35, target_sizes=target_sizes
)[0]
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(f"Detected {label.item()} with confidence " f"{round(score.item(), 3)} at location {box}")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 from transformers import AutoProcessor, GroundingDinoForObjectDetection                     │
│    2 from PIL import Image                                                                       │
│    3 import requests                                                                             │
│    4 import torch                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'GroundingDinoForObjectDetection' from 'transformers' 
(/home/xiaxinyuan/.conda/envs/modelscope/lib/python3.7/site-packages/transformers/__init__.py)